In [1]:
import torch
import non_local_boxes

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# 1. Basic Gradient Descent

In [ ]:
# Initialization
print("Initialization...")
starting_W = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)
nb_steps = 20
learning_rate = 2
tolerance=1e-06

P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR
external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)


# Gradient Descent
print("Iterating...")
W = torch.clone(starting_W)
for i in range(nb_steps):
    non_local_boxes.evaluate.phi_flat(W, P, Q).backward(gradient=external_grad)
    W2 = W - learning_rate*W.grad

    # check the tolerance

    W = W2

# print